In [20]:
import pandas as pd

In [21]:
df_fusionné = pd.read_csv('df_fusionné.csv')

In [22]:
df_fusionné = df_fusionné.sample(frac=1)


In [23]:
df_anormal = df_fusionné[df_fusionné["ColumnName"] != "N"]

In [24]:
value_counts = df_anormal["ColumnName"].value_counts()
print(value_counts)

ColumnName
Q    8043
V    7236
S    2781
F     803
Name: count, dtype: int64


In [16]:
X = df_anormal.iloc[:, 1:]
y = df_anormal.iloc[:, 0]

In [19]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [24]:
y_test

array([[0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [25]:
X_train.shape[1]

2160

In [26]:
y_train.shape[1]

4

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

In [37]:
model = Sequential([
    Conv1D(filters=64, kernel_size=6, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=3),
    BatchNormalization(),

    Conv1D(filters=128, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Conv1D(filters=128, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Conv1D(filters=64, kernel_size=6, activation='relu'),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),

    Flatten(),

    Dense(64, activation='relu'),

    Dense(32, activation='relu'),

    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Summary of the model
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 2155, 64)          448       
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 718, 64)           0         
 ng1D)                                                           
                                                                 
 batch_normalization_15 (Ba  (None, 718, 64)           256       
 tchNormalization)                                               
                                                                 
 conv1d_16 (Conv1D)          (None, 713, 128)          49280     
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 356, 128)          0         
 ng1D)                                                           
                                                      

In [38]:
from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Example for the medium complexity model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

Epoch 1/10
354/354 [==============================] - 14s 36ms/step - loss: 0.4498 - accuracy: 0.9100 - val_loss: 0.7126 - val_accuracy: 0.8031
Epoch 2/10
354/354 [==============================] - 12s 34ms/step - loss: 0.3175 - accuracy: 0.9443 - val_loss: 0.2777 - val_accuracy: 0.9465
Epoch 3/10
354/354 [==============================] - 12s 34ms/step - loss: 1.0670 - accuracy: 0.9250 - val_loss: 1.4973 - val_accuracy: 0.9335
Epoch 4/10
354/354 [==============================] - 12s 35ms/step - loss: 5.7875 - accuracy: 0.8975 - val_loss: 6.7188 - val_accuracy: 0.9316
Epoch 5/10
354/354 [==============================] - 12s 35ms/step - loss: 31.8966 - accuracy: 0.8734 - val_loss: 16.1506 - val_accuracy: 0.8741
Epoch 6/10
354/354 [==============================] - 12s 35ms/step - loss: 308.8495 - accuracy: 0.7129 - val_loss: 29.9282 - val_accuracy: 0.8301
Epoch 7/10
354/354 [==============================] - 12s 35ms/step - loss: 38.4793 - accuracy: 0.7992 - val_loss: 28.1875 - val_ac